## StableDiffusion Model 

설치 라이브러리

In [ ]:
pip install openai==1.10.0

In [ ]:
pip install diffusers accelerate safetensors transformers

In [ ]:
pip install compel

프롬프트 생성 클래스 

In [2]:
import requests
import pandas as pd
import json
from datetime import datetime
import hmac, hashlib
from pytz import timezone
from keybert import KeyBERT
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import openai

# gpt api 관련 변수
openai.api_key = 'GPT KEY'

# papago api 관련 변수
papago_client_id = "파파고 클라이언트"  # 개발자 센터에서 발급받은 클라이언트 ID
papago_client_secret = '파파고 시크릿'  # 개발자 센터에서 발급받은 클라이언트 시크릿
papago_url = 'https://openapi.naver.com/v1/papago/n2mt'
papago_headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Naver-Client-Id': papago_client_id,
    'X-Naver-Client-Secret': papago_client_secret,
}


class PromptEnglish:
    def __init__(self, text):
        self.text = text
        
    def gpt(self):
        input_message = f'"{self.text}"와 유사한 문장 10개만 생성해줘. 디테일한 내용이 더 추가되어도 괜찮아'
        
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {"role": "system", "content": "You are helpful assistant."},
                    {"role": "user", "content": input_message}
                ],
                temperature=0
            )
            improved_text = response.choices[0].message.content
            print("improved_text", improved_text)
            
            sentences = [sentence.strip() for sentence in improved_text.split('\n')]
            # print(sentences)            
            augments = []
            for sentence in sentences:
                temp = sentence.split('.')
                hap = ''
                for i in range(len(temp)):
                    if i == 0:
                        pass
                    elif i == len(temp)-1:
                        hap += temp[i] 
                    else:
                        hap = hap + temp[i] + '.'
                augments.append(hap)

            # 입력값도 추가
            augments.append(self.text)
            return augments
        
        except Exception as e:
            print(f"An error occurred: {e}")
            return str(e)

    @classmethod
    def papago_translation(cls, text_list):
        translated_texts = []
        
        # 한국어, 영어로 변환
        for text in text_list:
            data = {
                'source': 'ko',
                'target': 'en',
                'text': text,
            }

            response = requests.post(papago_url, headers=papago_headers, data=data)
            result = response.json()

            if 'errorMessage' in result:
                return result['errorMessage']

            else:
                translated_text = result['message']['result']['translatedText']
                translated_texts.append(translated_text)
            
            base_prompt = translated_texts[-1]
                
        return translated_texts, base_prompt
    
    # BERT모델을 활용한 키워드, 가중치 추출
    @classmethod
    def bert(cls, text_list):
        bow = []
        
        extractor = KeyBERT("all-mpnet-base-v2")
        
        for i in range(len(text_list)):
            keywords = extractor.extract_keywords(text_list[i], stop_words='english')
            bow.append(keywords)
            
        new_bow = []
        
        for i in range(0, len(bow)):
            for j in range(len(bow[i])):
                new_bow.append(bow[i][j])
                
        df = pd.DataFrame(new_bow, columns=['keyword', 'weight'])
        
        # GROUPBY
        # 동일 keyword에 대해서 가중치 중앙값 추출
        weight_df = df.groupby('keyword').agg('median').sort_index()
        # 동일 keyword의 count 추출 후, 칼럼명 count로 변환
        count_df = df.groupby('keyword').agg('count').sort_index()
        count_df.rename(columns={'weight':'count'}, inplace=True)
        
        # keyword기준 두 df 합치기
        result_df = weight_df.join(count_df)
        result_df.sort_values('count', ascending=False, inplace=True)
        
        return result_df
    
    @classmethod
    ## +로 가중치 부여한 프롬프트 
    def prompt_weighting(cls, dataframe, text_list):
        
        # 중요도 높은 단어 상위 5개 추출
        top_weight_word = dataframe.head(5).index.values
        
        weight_dict = {}
        prompt = ""
        
        # 단어:가중치 형태의 딕셔너리 생성
        for i in range(len(top_weight_word)):
            weight_dict[top_weight_word[i]] = '+' * (5-i)

        # 단어 옆에 가중치 추가
        for text in text_list:
            indexs = []
            for key, val in weight_dict.items():
                if key in text:
                    index = text.find(key)
                    
                    if index != -1:
                        text = text[:index + len(key)] + val + text[index + len(key):]
            prompt += text
        
        # 마침표 제거. 쉼표로 프롬포트 연결
        if "." in prompt:
            prompt.replace(".", ", ")
        
        return prompt[:-2]

    @classmethod 
    def positive_prompt(cls, df):
        result = ''
        keyword = df.index.values
        
        for i in range(len(keyword)):
            if i != len(keyword)-1:
                result = result + keyword[i] + '+++, '
            else:
                result = result + keyword[i] + '+++'
                
        return result

/tmp/ipykernel_12326/4087964187.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-01-31 09:07:16.377593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:07:16.377631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:07:16.378255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable 

prompt 추출

In [ ]:
# 프롬포트 예시
input = '학사복을 입고 꽃다발을 들고 있는 금발 머리의 여자는 교문 앞에서 부모님과 함께 사진을 찍고 있고, 교문 뒤에는 학교의 상징인 큰 시계탑이 있다.'
myprompt = PromptEnglish(input)

result = myprompt.myGPT()
translated, base_prompt = myprompt.papago_translation(result)
print('prompt: ', base_prompt)

df = myprompt.bert(translated)
print(df)

prompt_2 = myprompt.positive_prompt(df)
print('prompt_2: ', prompt_2)


prompt: 'A blond woman wearing a school uniform and holding a bouquet of flowers is taking a picture with her parents in front of the school gate, and behind the school gate is a large clock tower, the symbol of the school.'

prompt_2: 'school+++, graduation+++, commemorative+++, graduates+++, garden+++, blond+++, students+++, hat+++, gate+++, birds+++, ceremony+++, celebrate+++, blonde+++, women+++, flowers+++, gathering+++, building+++, lake+++, parents+++, photo+++, photos+++, picture+++, bouquet+++, tower+++, uniform+++'


이미지 저장 시에 사용하는 함수


In [ ]:
from datetime import datetime
import pytz

def generate_filename(model):
    korea_tz = pytz.timezone('Asia/Seoul')
    utc_now = datetime.utcnow()
    korea_now = utc_now.replace(tzinfo=pytz.utc).astimezone(korea_tz)
    korea_now = korea_now.strftime('%H:%M')
    # file_name = "/tf/notebook/SD_result_img/" + model + str(korea_now) + ".png"
    return "/tf/notebook/SD_result_img/" + model + str(korea_now) + ".png"

이미지 생성


In [ ]:
from diffusers import  StableDiffusionXLPipeline, DDIMScheduler, DDPMScheduler
from diffusers.utils import load_image
import torch, gc
import pytz
gc.collect()
torch.cuda.empty_cache()
print('now cache is empty')

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16, 
    variant="fp16", 
    use_safetensors=True
)
pipe.to('cuda')
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

positive_prompt = 'realism++, realistic++, super-detailed++, true-to-life++, best quality++, balanced++, well-defined++, Modernist++, well-defined facial features++, detailed faces++'
negative_prompt = 'CG, wallpaper, animation, anime, doll, disney, cartoons, cropped, misshapen, blurry, pixiv, artstation, desaturated, abstract, surreal, pixelated, noisy, pop art, no faces, no objects, no landscape, mutilated, disfigured, ugly, deformed, clear visibility of faces, no abstract faces, shadows on faces'

prompt_2 = prompt_2 + positive_prompt

n = 120
m = 7.5

# generator=torch.manual_seed(1)
image = pipe(prompt=base_prompt, prompt_2=prompt_2, negative_prompt=negative_prompt, num_inference_steps=n, guidance_scale=m).images[0]
 
# # file_name = generate_filename('sdxl-base-1.0-DDIM', n, m)
# # image.save(file_name)
# image